For the avila dataset, some measures available in Quantus are computed in the results files for every ranking.

In order to get the value for $e^{inv}$, we need to know what index corresponds to the inv of each ranking.

This should have been computed in the original file but, since it wasn't, it's now easier to compute the indices instead of the whole thing.

**It seems that the inverse is simply the same index but starting from the end, but we'll do it this way just to be sure**

In [45]:
import numpy as np
import itertools
import os

PROJ_DIR = os.path.realpath(os.path.dirname(os.path.abspath('')))

# Avila dataset
DATASET = 'avila'
DATASET_PATH = os.path.join(PROJ_DIR,'assets', 'data', f'{DATASET}.npz')
file_data = np.load(DATASET_PATH)
x_train = file_data['x_train']
NUM_VARS = x_train.shape[1]

permutations = list(itertools.permutations(range(NUM_VARS)))
shape = tuple([NUM_VARS for x in range(NUM_VARS)])
#lookup = -1 * np.ones(shape, dtype=int) # This was a memory behemoth
lookup = {}

result = -1 * np.ones(len(permutations), dtype=int)

In [46]:
def add_to_lookup(index, value, lookup):
    for i in index[:-1]:
        if i not in lookup:
            lookup[i] = {}
        lookup = lookup[i]
    lookup[index[-1]] = value

def check_lookup(index, lookup):
    for i in index[:-1]:
        if i not in lookup:
            return -1
        lookup = lookup[i]
    return lookup[index[-1]]


In [47]:
from tqdm import tqdm

for i,p in tqdm(enumerate(permutations)):
    #lookup[p] = i
    add_to_lookup(p, i, lookup)
    #rev = tuple(reversed(p))
    rev = tuple([NUM_VARS-1-i for i in p])
    v = check_lookup(rev, lookup)
    if v>=0:
        result[v] = i
        result[i] = v
    #if lookup[rev] >= 0:
        #result[lookup[rev]] = i
        #result[i] = lookup[rev]

3628800it [00:07, 484361.19it/s]


In [51]:
np.savez(os.path.join(PROJ_DIR, 'results', f'{DATASET}_permutations_inv_lookup.npz'),
         inv_lookup=result)